In [ ]:
%load_ext watermark


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp

from pylib._seed_global_rngs import seed_global_rngs


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "2025-04-30-profile-mut-screen"
teeplot_subdir


In [ ]:
seed_global_rngs(1)


## Get Data


In [ ]:
df = pd.read_parquet("https://osf.io/8pgy5/download")


In [ ]:
df["replicate_uuid"].nunique()


In [ ]:
df["is_focal_mutation"] = df["mut_char_pos"] == 0


In [ ]:
records = []
for (trt_name, replicate_uuid, screen_uuid, screen_name, tb_stat), grp in df[
    (df["trt_hsurf_bits"] == 0)
    & (df["trt_n_downsample"] == df["trt_n_downsample"].max())
    & (df["tb_stat"].str.contains("ratio"))
].groupby(
    ["trt_name", "replicate_uuid", "screen_uuid", "screen_name", "tb_stat"],
    observed=True,
):
    grp_focal = grp[grp["is_focal_mutation"]]
    grp_nonfocal = grp[~grp["is_focal_mutation"]]

    record = {
        "trt_name": trt_name,
        "replicate_uuid": replicate_uuid,
        "screen_uuid": screen_uuid,
        "screen_name": screen_name,
        "tb_stat": tb_stat,
    }

    records.append(
        {
            **record,
            "binom_p": (
                np.nan if len(grp_focal) == 0 else grp_focal["binom_p"].item()
            ),
            "mutation": "focal",
        },
    )
    records.append(
        {
            **record,
            "binom_p": grp_nonfocal["binom_p"].sample(1).item(),
            "mutation": "nonfocal",
        },
    )


In [ ]:
with tp.teed(
    sns.catplot,
    data=pd.DataFrame(records),
    x="trt_name",
    y="binom_p",
    hue="mutation",
    row="screen_name",
    col="tb_stat",
    kind="box",
    height=2,
    aspect=1.5,
    margin_titles=True,
):
    pass
